In [ ]:
# !!! only run when using colab !!!
import os
! git clone https://github.com/Plant-Root-Soil-Interactions-Modelling/CPlantBox # downloading the source code
os.chdir("/content/CPlantBox/")
! cmake .
! make
os.chdir("/content/CPlantBox/tutorial/jupyter/workshop_11_2_2022/examples_usage/")

# Workshop 11.2.2022 - test case

## Presentation of photosynthesis module

In [1]:
import sys; sys.path.append("../../../.."); sys.path.append("../../../../src/python_modules")
from xylem_flux import XylemFluxPython  # Python hybrid solver
from Leuning import Leuning
import plantbox as pb
import vtk_plot as vp
import pandas as pd
from matplotlib.ticker import MaxNLocator
import numpy as np
import matplotlib.pyplot as plt

define parameters:

In [2]:
kz = 4.32e-1  # axial conductivity [cm^3/day] 
kr = 1.728e-4  # radial conductivity of roots [1/day]
kr_stem = 1.e-20  # radial conductivity of stem  [1/day], set to almost 0
gmax =  0.004 #  cm3/day radial conductivity of leaves = stomatal conductivity [1/day]
p_a =  -1000  #static air water potential 
simtime = 14.0  # [day] for task b
k_soil = []
plotResults = True
saveResults = False

t_init = 70
t_end = 90

set up root system

In [3]:
# root system 
pl = pb.MappedPlant() #pb.MappedRootSystem() #pb.MappedPlant()
path = "../../../../modelparameter/plant/" #"../../../../modelparameter/rootsystem/" 
name = "manyleaves" #"Anagallis_femina_Leitner_2010"  # Zea_mays_1_Leitner_2010
pl.readParameters(path + name + ".xml")

#load data
df = pd.read_csv('../../../../modelparameter/Selhausen_weather_data.txt', delimiter = "\t")

""" soil """
min_ = np.array([-5, -5, -15])
max_ = np.array([9, 4, 0])
res_ = np.array([5, 5, 5])
pl.setRectangularGrid(pb.Vector3d(min_), pb.Vector3d(max_), pb.Vector3d(res_), True)  # cut and map segments


pl.initialize()
pl.simulate(simtime, False)


r = Leuning(pl) 
r.setKr([[kr],[kr_stem],[gmax]]) #gmax will be changed by the leuning function 
r.setKx([[kz]])
r.airPressure = p_a
leaf_nodes = r.get_nodes_index(4)


OrganRandomParameter::readXML: warning! parameter geometryN is defined in the xml, but not available in organ leaf
OrganRandomParameter::readXML: warning! parameter leafGeometry is defined in the xml, but not available in organ leaf


launch computation loop

In [4]:
# Numerical solution 
results=[]
resultsAn=[]
resultsgco2=[]
resultsVc=[]
resultsVj=[]
resultscics=[]
resultsfw=[]
resultspl=[]

for i in range(t_init, t_end):
    
    Q_input = df['PAR'][i]
    RH_input = df['RH'][i]
    Tair_input = df['Tair'][i]
    p_s_input = df['ps'][i]
    N_input = 4.4 #nitrogen satisfaction for small wheat plants
    cs_input = df['co2'][i]
    var = [Q_input,RH_input, Tair_input, p_s_input, N_input, cs_input]
    es = 0.61078 * np.exp(17.27 * var[2] / (var[2] + 237.3)) #FAO56
    ea = es * var[1]
    VPD = es - ea 
    r.Param['Patm']=df['Pair'][i]
    rx = r.solve_leuning( sim_time = simtime,sxx=[var[3]], cells = True, Qlight = var[0],VPD = VPD,Tl = var[2] + 273.15,p_linit = var[3],
    ci_init = var[5]*0.7, cs =var[5], soil_k = [], N = var[4], log = False, verbose = False)
    fluxes = r.radial_fluxes(simtime, rx, [var[3]], k_soil, True)  # cm3/day
    organTypes = np.array(r.rs.organTypes)
    results.append(sum(np.where(organTypes == 4, fluxes,0)))
    resultsAn.append(np.mean(r.An)*1e6)
    resultsVc.append(np.mean(r.Vc)*1e6)
    resultsVj.append(np.mean(r.Vj)*1e6)
    resultsgco2.append(np.mean(r.gco2))
    resultscics.append(np.mean(r.ci)/var[5])
    resultsfw.append(np.mean(r.fw))
    resultspl.append(np.mean(r.x[leaf_nodes]))

Plot results:

In [5]:
timePlot = df[t_init:t_end]['time']
if plotResults:
    fig, axs = plt.subplots(2,2)
    axs[0, 0].plot(timePlot, resultsAn)
    axs[0, 0].set(xlabel='', ylabel='mean An (μmol CO2 m-2 s-1)')
    axs[0, 0].xaxis.set_major_locator(MaxNLocator(5))
    axs[1, 0].plot(timePlot, resultsVc, 'tab:red')
    axs[1, 0].set(xlabel='time', ylabel='mean Vc (μmol CO2 m-2 s-1)')
    axs[1, 0].xaxis.set_major_locator(MaxNLocator(5))
    axs[0, 1].plot(timePlot, resultsVj, 'tab:brown')
    axs[0, 1].set(xlabel='', ylabel='mean Vj (μmol CO2 m-2 s-1)')
    axs[0, 1].xaxis.set_major_locator(MaxNLocator(5))
    axs[1, 1].plot(timePlot, resultsgco2, 'tab:brown')
    axs[1, 1].set(xlabel='time', ylabel='mean gco2 (mol CO2 m-2 s-1)')
    axs[1, 1].xaxis.set_major_locator(MaxNLocator(5))
    plt.show()
    fig, axs = plt.subplots(2,2)
    axs[0, 0].plot(timePlot, results)
    axs[0, 0].set(xlabel='', ylabel='E')
    axs[0, 0].xaxis.set_major_locator(MaxNLocator(5))
    axs[0, 1].plot(timePlot, resultsfw, 'tab:brown')
    axs[0, 1].set(xlabel='', ylabel='fw')
    axs[0, 1].xaxis.set_major_locator(MaxNLocator(5))
    axs[1, 0].plot(timePlot, resultspl, 'tab:brown')
    axs[1, 0].set(xlabel='time', ylabel='pl')
    axs[1, 0].xaxis.set_major_locator(MaxNLocator(5))
    axs[1, 1].plot(timePlot, resultscics, 'tab:brown')
    axs[1, 1].set(xlabel='time', ylabel='ci/cs (-)')
    axs[1, 1].xaxis.set_major_locator(MaxNLocator(5))
    plt.show()